# 문서 군집화 - Opinion Review dataset

In [2]:
import pandas as pd
import glob, os

path = 'OpinosisDataset1.0/topics'
os.path.join(path, '*.data')  # topic 폴더 안의 파일이름 끝이 .data로 끝나는 것을 볼 수 있다. (*.data는 .data로 끝나는 모든것이라는 의미)
                              # Windows - \\, Mac - /

'OpinosisDataset1.0/topics\\*.data'

In [4]:
all_files = glob.glob(os.path.join(path, '*.data'))
all_files[:5]

['OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data']

In [5]:
file = all_files[0]
file

'OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data'

In [11]:
file.split('\\')[-1].split('.')[0]

'accuracy_garmin_nuvi_255W_gps'

* with 함수란?

In [ ]:
open()  # 괄호 안에는 file_name이 들어간다

read(), write()

close()

# 사람들이 보통 open, read, write만 하고 close 안하는데 with 한 다음 open()을 쓰면 with 문을 벗어나는 순간에 자동 close

In [18]:
filename_list = []
opinion_text = []
for file in all_files:
    with open(file, encoding = 'latin1') as f:
        text = f.read()
    # 위의 with문은 
    # f = open(file)
    # text = f.read()
    # f.close 와 같다
    opinion_text.append(text)
    filename = file.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

df = pd.DataFrame({'filename' : filename_list, 'opinion' : opinion_text})
df.head()

,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ..."


In [17]:
# filename_list1 = [file.split('\\')[-1].split('.')[0] for file in all_files]

* Simple tokenizer 함수

In [19]:
from nltk import word_tokenize

def simple_tokenizer(text):             # 글자수가 2개 이하인 토큰은 제거
    # word_list = word_tokenize(text)
    # word_list = [word for word in word_list if len(word) > 2]
    # return word_list
    return [word for word in word_tokenize(text) if len(word) > 2]

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(tokenizer = simple_tokenizer, stop_words='english', 
                        ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# 비지도학습에서는 train/test 분리한 적 없으므로 fit, transform 한방에 해도 된다
feature = tvect.fit_transform(df.opinion)

* 군집화

In [21]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, max_iter=10000, random_state=2022)

# 비지도학습이므로 X만 훈련시킨다
kmeans.fit(feature)

KMeans(max_iter=10000, n_clusters=5, random_state=2022)

In [1]:
df['cluster_label'] = kmeans.labels_
df.head()

NameError: name 'kmeans' is not defined

In [24]:
df['cluster'].value_counts()

2    16
3    13
4    10
0     7
1     5
Name: cluster, dtype: int64

In [31]:
# 3개의 집합으로 군집화
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=2022)
kmeans.fit(feature)
df['cluster_label'] = kmeans.labels_
df['cluster_label'].value_counts()

0    25
1    16
2    10
Name: cluster_label, dtype: int64

- 군집별 핵심 단어 추출하기

In [26]:
feature.shape  # bigram까지 포함한 1단어, 2단어 갯수가 4154개

(51, 4154)

In [27]:
cluster_centers = kmeans.cluster_centers_
cluster_centers.shape

(3, 4154)

In [32]:
from cluster import get_cluster_details

feature_names = tvect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans, cluster_data = df, feature_names=feature_names,
                                      clusters_num=3, top_n_features=10)

In [33]:
for cluster_num, cluster_detail in cluster_details.items():
    print(f'####### Cluster {cluster_num}')
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('==================================================')

####### Cluster 0
Top features: ['screen', 'battery', 'battery life', 'keyboard', 'kindle', 'life', 'directions', 'size', 'voice', 'speed']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['hotel', 'service', 'rooms', 'staff', 'room', 'food', 'location', 'clean', 'bathroom', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['interior', 'mileage', 'seats', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_hond